<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_10_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import json
from tqdm import tqdm

In [21]:
class PotentialClassifier:
    def __init__(self, model_path='potential_model.json'):
        self.model_path = model_path
        self.classes = []      # Список классов
        self.potentials = {}   # Потенциалы для каждого класса
        self.alpha = 0.01      # Скорость обучения
        self.max_epochs = 3    # Максимальное число эпох
        self.feature_size = None  # Размерность признакового пространства

    def extract_features(self, image):
        """Извлечение признаков из изображения"""
        return image.flatten()

    def calculate_potential(self, x, class_name):
        """Вычисление потенциала для данного класса"""
        potential = 0.0
        if class_name in self.potentials:
            for pattern in self.potentials[class_name]:
                distance = np.linalg.norm(x - pattern['x'])
                potential += pattern['weight'] * np.exp(-distance)
        return potential

    def train(self, dataset_path):
        """Режим обучения"""
        print("Начало обучения...")

        # Инициализация
        self.classes = sorted([d for d in os.listdir(dataset_path)
                            if os.path.isdir(os.path.join(dataset_path, d))])
        self.potentials = {class_name: [] for class_name in self.classes}

        # Загрузка данных
        X, y = [], []
        for class_name in self.classes:
            class_path = os.path.join(dataset_path, class_name)
            print(f"Загрузка класса {class_name}...")

            for img_file in tqdm(os.listdir(class_path)):
                if img_file.lower().endswith(('.png', '.bmp', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_file)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                    if img is not None:
                        # Нормализация и извлечение признаков
                        img_normalized = img / 255.0
                        features = self.extract_features(img_normalized)
                        X.append(features)
                        y.append(class_name)

        # Установка размерности признаков
        if X:
            self.feature_size = len(X[0])

        # Обучение
        for epoch in range(self.max_epochs):
            errors = 0

            for x, true_class in zip(X, y):
                potentials = {c: self.calculate_potential(x, c) for c in self.classes}
                predicted = max(potentials, key=potentials.get)

                if predicted != true_class:
                    errors += 1
                    self.potentials[true_class].append({'x': x, 'weight': self.alpha})
                    self.potentials[predicted].append({'x': x, 'weight': -self.alpha})

            print(f"Эпоха {epoch+1}, ошибок: {errors}/{len(X)}")
            if errors == 0:
                break

        self.save_model()
        print("Обучение завершено!")

    def save_model(self):
        """Сохранение модели в файл"""
        model_data = {
            'feature_size': self.feature_size,
            'classes': self.classes,
            'potentials': {
                c: [{'x': p['x'].tolist(), 'weight': p['weight']}
                   for p in self.potentials[c]]
                for c in self.classes
            }
        }

        with open(self.model_path, 'w') as f:
            json.dump(model_data, f, indent=2)
        print(f"Модель сохранена в {self.model_path}")

    def load_model(self):
        """Загрузка модели из файла"""
        try:
            with open(self.model_path, 'r') as f:
                model_data = json.load(f)

            self.feature_size = model_data['feature_size']
            self.classes = model_data['classes']
            self.potentials = {
                c: [{'x': np.array(p['x']), 'weight': p['weight']}
                   for p in model_data['potentials'][c]]
                for c in model_data['classes']
            }
            #print(f"Модель загружена из {self.model_path}")
            return True
        except Exception as e:
            print(f"Ошибка загрузки модели: {str(e)}")
            return False

    def recognize(self, image_path):
        """Режим распознавания"""
        if not self.load_model():
            return None

        # Загрузка изображения
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("Не удалось загрузить изображение")
            return None

        # Нормализация и извлечение признаков
        img_normalized = img / 255.0
        features = self.extract_features(img_normalized)

        # Вычисление потенциалов
        potentials = {c: self.calculate_potential(features, c) for c in self.classes}
        return max(potentials, key=potentials.get)

    def evaluate(self, test_path):
      """Оценка точности на тестовом наборе"""
      if not self.load_model():
          return None

      correct = 0
      total = 0
      class_stats = {c: {'correct': 0, 'total': 0} for c in self.classes}

      print("\nНачало оценки точности...")
      for class_name in self.classes:
          class_path = os.path.join(test_path, class_name)
          if not os.path.exists(class_path):
              continue

          for img_file in tqdm(os.listdir(class_path), desc=f"Класс {class_name}"):
              if img_file.lower().endswith(('.png', '.bmp', '.jpg', '.jpeg')):
                  img_path = os.path.join(class_path, img_file)
                  prediction = self.recognize(img_path)

                  if prediction == class_name:
                      correct += 1
                      class_stats[class_name]['correct'] += 1
                  total += 1
                  class_stats[class_name]['total'] += 1

      if total == 0:
          print("Нет данных для оценки")
          return 0.0

      accuracy = correct / total
      print("\nРезультаты оценки:")
      print(f"Общая точность: {accuracy:.2%}")
      print("\nДетализация по классам:")
      for cls, stats in class_stats.items():
          if stats['total'] > 0:
              cls_acc = stats['correct'] / stats['total']
              print(f"{cls}: {cls_acc:.2%} ({stats['correct']}/{stats['total']})")

      return accuracy

In [13]:
classifier = PotentialClassifier()
classifier.train("processed_dataset/train")

Начало обучения...
Загрузка класса 0...


100%|██████████| 66/66 [00:00<00:00, 8043.47it/s]


Загрузка класса 1...


100%|██████████| 66/66 [00:00<00:00, 8154.36it/s]


Загрузка класса 10...


...

Загрузка класса Y...


100%|██████████| 64/64 [00:00<00:00, 5510.32it/s]


Загрузка класса Z...


100%|██████████| 66/66 [00:00<00:00, 4779.83it/s]


Эпоха 1, ошибок: 871/2360
Эпоха 2, ошибок: 651/2360
Эпоха 3, ошибок: 95/2360
Модель сохранена в potential_model.json
Обучение завершено!


In [ ]:
classifier = PotentialClassifier()
accuracy = classifier.evaluate("processed_dataset/test")
print(f"\nИтоговая точность модели: {accuracy:.2%}")


Начало оценки точности...


Класс 1:  48%|████▊     | 15/31 [00:56<01:01,  3.84s/it]

In [2]:
!unzip processed_dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: processed_dataset/train/X/0017.png  
  inflating: processed_dataset/train/X/0057.png  
  inflating: processed_dataset/train/X/0001.png  
  inflating: processed_dataset/train/X/0051.png  
 extracting: processed_dataset/train/X/0003.png  
  inflating: processed_dataset/train/X/0004.png  
  inflating: processed_dataset/train/X/0049.png  
  inflating: processed_dataset/train/X/0010.png  
  inflating: processed_dataset/train/X/0007.png  
  inflating: processed_dataset/train/X/0023.png  
  inflating: processed_dataset/train/X/0018.png  
 extracting: processed_dataset/train/X/0009.png  
  inflating: processed_dataset/train/X/0013.png  
  inflating: processed_dataset/train/X/0058.png  
  inflating: processed_dataset/train/X/0016.png  
  inflating: processed_dataset/train/X/0047.png  
  inflating: processed_dataset/train/X/0045.png  
  inflating: processed_dataset/train/X/0054.png  
  inflating: processed_dataset/t